In [1]:
import import_ipynb

#import model
%cd ..
%cd "protein-ppi-encoding-module"
from transformerGO import *

#import dataset functions
%cd ..
%cd "datasets"
from dataset_manip import *
%cd .. 

%cd "training-testing"
from training_helper import * 
%cd ..

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim

import time
import pickle

import random
from random import shuffle
from sklearn.metrics import roc_auc_score
from torchsummary import summary

from sklearn import metrics
import matplotlib.pyplot as plt

from tqdm import tqdm as prog_bar

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device available: ", device, " ", torch.cuda.get_device_name(0))

#for np array from nested sequences
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

C:\Users\Ieremie\Desktop\TransformerGO
C:\Users\Ieremie\Desktop\TransformerGO\protein-ppi-encoding-module
importing Jupyter notebook from transformerGO.ipynb
importing Jupyter notebook from harvard_transformer.ipynb
C:\Users\Ieremie\Desktop\TransformerGO
C:\Users\Ieremie\Desktop\TransformerGO\datasets
importing Jupyter notebook from dataset_manip.ipynb
C:\Users\Ieremie\Desktop\TransformerGO
C:\Users\Ieremie\Desktop\TransformerGO\training-testing
importing Jupyter notebook from training_helper.ipynb
C:\Users\Ieremie\Desktop\TransformerGO
Device available:  cuda   GeForce GTX 960M


In [2]:
def helper_collate(batch):
    MAX_LEN_SEQ = get_max_len_seq(batch)
    return transformerGO_collate_fn(batch, MAX_LEN_SEQ, EMB_DIM, pytorch_pad = False)

In [3]:
## STRINGDB BENCHMARK DATASET ##
EMB_DIM = 64
organism = 9606
data_path = 'datasets/onto2vec-datasets-string/data'

go_embed_pth = f"term-encoding-module/emb/go-terms-{EMB_DIM}.emd"
go_id_dict_pth = "term-encoding-module/go_id_dict"
protein_go_anno_pth = data_path +"/goa_human.gaf.gz"
alias_path = data_path + f'/{organism}.protein.aliases.v11.0.txt.gz'

neg_path_train = data_path + f'/train/{organism}.no-mirror.negative_interactions.txt'
poz_path_train = data_path + f'/train/{organism}.no-mirror.protein.links.v11.0.txt'
neg_path_valid = data_path + f'/valid/{organism}.no-mirror.negative_interactions.txt'
poz_path_valid = data_path + f'/valid/{organism}.no-mirror.protein.links.v11.0.txt'
neg_path_test = data_path + f'/test/{organism}.no-mirror.negative_interactions.txt'
poz_path_test = data_path + f'/test/{organism}.no-mirror.protein.links.v11.0.txt'

#the splitting for StringDB is already done, just load the data
train_set = get_dataset_split_stringDB(poz_path_train, neg_path_train, protein_go_anno_pth, go_id_dict_pth, go_embed_pth, shuffle,alias_path, ratio = [1, 0, 0], stringDB = True)[0]
valid_set = get_dataset_split_stringDB(poz_path_valid, neg_path_valid, protein_go_anno_pth, go_id_dict_pth, go_embed_pth, shuffle,alias_path, ratio = [1, 0, 0], stringDB = True)[0]
test_set = get_dataset_split_stringDB(poz_path_test, neg_path_test, protein_go_anno_pth, go_id_dict_pth, go_embed_pth, shuffle,alias_path, ratio = [1, 0, 0], stringDB = True)[0]

params = {'batch_size': 32,'collate_fn': helper_collate}

Rejected interactions where at least one protein has no annotation:  6573
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 500]:  0
Number of interactions: 262570
Rejected interactions where at least one protein has no annotation:  38444
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 500]:  0
Number of interactions: 230699
Rejected interactions where at least one protein has no annotation:  1657
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 500]:  0
Number of interactions: 65628
Rejected interactions where at least one protein has no annotation:  9620
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 500]:  0
Number of interactions: 57665
Rejected interactions where at least one protein has no annotation:  2018
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 500]:  0
Number of interactions: 82088
Rejected interactions where at least one protein has no annotation:  11801
Re

In [8]:
#TRANSFORMERGO DATASET#
organism = 9606
EMB_DIM = 64
data_path = 'datasets/transformerGO-dataset/'
go_embed_pth = data_path + f"go-terms/emb/go-terms-{EMB_DIM}.emd"
go_id_dict_pth = data_path + "go-terms/go_id_dict"
protein_go_anno_pth = data_path +"stringDB-files/goa_human.gaf.gz"
alias_path = data_path + f'stringDB-files/{organism}.protein.aliases.v11.5.txt.gz'

neg_path = data_path + f'interaction-datasets/{organism}.protein.negative.v11.5.txt'
poz_path= data_path + f'interaction-datasets/{organism}.protein.links.v11.5.txt'

#choosing 16% data for validation to be similar with opa2vec where validation is 20% of training data (80%)
intr_set_size_filter = [0,5000]
go_filter = 'ALL'
train_set, valid_set, test_set, _ = get_dataset_split_stringDB(poz_path, neg_path, protein_go_anno_pth, go_id_dict_pth, go_embed_pth,\
                                        shuffle, alias_path, ratio = [0.64, 0.16, 0.2], go_filter = go_filter, intr_set_size_filter = intr_set_size_filter, max_intr_size = None)

params = {'batch_size': 32,'collate_fn': helper_collate}

Rejected interactions where at least one protein has no annotation:  9424
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 5000]:  0
Number of interactions: 243560
Rejected interactions where at least one protein has no annotation:  10385
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 5000]:  0
Number of interactions: 242599


In [3]:
## JAINS TCSS DATASETS ##
EMB_DIM = 64
neg_path = "datasets/jains-TCSS-datasets/human_data/iea-/negatives.human.f"
poz_path = "datasets/jains-TCSS-datasets/human_data/iea-/positives.human.f"

go_embed_pth = "term-encoding-module/emb/go-terms-64.emd"
go_id_dict_pth = "term-encoding-module/go_id_dict"
protein_go_anno_pth = "datasets/jains-TCSS-datasets/human_data/gene_association.goa_human"

train_set, valid_set, test_set, all_dataset = get_dataset_split_stringDB(poz_path, neg_path, protein_go_anno_pth, go_id_dict_pth, go_embed_pth, shuffle, "", ratio = [0.8, 0.2, 0], stringDB = False)
params = {'batch_size': 16,'collate_fn': helper_collate}

<string>:153: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.


Rejected interactions where at least one protein has no annotation:  0
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 500]:  0
Number of interactions: 1288
Rejected interactions where at least one protein has no annotation:  0
Rejected interactions where go_filter=ALL and intr_set_size_filter=[0, 500]:  0
Number of interactions: 1288


In [4]:
print("Train set: ", len(train_set), '\n', "Valid set: ", len(valid_set), '\n', "Test set: ", len(test_set), '\n')
train_grt = data.DataLoader(train_set, **params, shuffle = True)
val_grt = data.DataLoader(valid_set, **params, shuffle = True)
test_grt = data.DataLoader(test_set, **params, shuffle = False)

Train set:  2060 
 Valid set:  515 
 Test set:  1 



In [7]:
#TRANSFORMERGO MODEL #
MODEL_SIZE = EMB_DIM
NR_HEADS = 8
NR_LAYERS = 3
DROPOUT = 0.2
SIZE_FF = 4 * MODEL_SIZE
LR = 0.0001

model = TransformerGO_Scratch(MODEL_SIZE, NR_HEADS, NR_LAYERS, SIZE_FF, DROPOUT)
#model = TransformerGO(MODEL_SIZE, NR_HEADS, NR_LAYERS, SIZE_FF, DROPOUT)

In [6]:
#SIMPLE FEED FORWARD NEURAL NETORK#
MODEL_SIZE = EMB_DIM
DROPOUT = 0.2
LR = 0.0001
F_1 = 200
F_2 = 400
F_3 = 200
F_4 = 1

model =  GO_Sum_NN(MODEL_SIZE, F_1, F_2, F_3, F_4, DROPOUT)
model = model.to(device)

In [8]:
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=LR) 
criterion = nn.BCEWithLogitsLoss().to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

350529


In [9]:
def train(model, iterator, optimizer, criterion,  torch_vers = False):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    pred = []
    lab = []
    for batch in prog_bar(iterator):
        optimizer.zero_grad()
        
        #padded pairs: tensor of shape N * 2(protein pair) * L(longest seq) * Emb dim
        padded_pairs = batch[0].to(device)
        labels = batch[1].to(device)
        mask = batch[2].to(device)
        
        #split data into protA and protB
        gosetA_batch = padded_pairs[:,0]
        gosetB_batch = padded_pairs[:,1]
        
        #permute the data to fit the pytorch transformer
        if torch_vers:
            gosetA_batch = gosetA_batch.permute(1,0,2)
            gosetB_batch = gosetB_batch.permute(1,0,2)
        
        predictions = model(gosetA_batch, gosetB_batch, mask[:,0], mask[:,1]).squeeze(1)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)
        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        pred = pred + list(predictions.cpu().data.numpy())
        lab = lab + list(labels.cpu().data.numpy())
 
    return epoch_loss / len(iterator), epoch_acc / len(iterator), roc_auc_score(lab,pred)

In [10]:
def evaluate(model, iterator, criterion, torch_vers = False):
    
    epoch_loss = 0
    epoch_acc = 0
    pred = []
    lab = []
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            
            #padded pairs: tensor of shape N * 2(protein pair) * L(longest seq) * Emb dim
            padded_pairs = batch[0].to(device)
            labels = batch[1].to(device)
            mask = batch[2].to(device)
        
            #split data into protA and protB
            gosetA_batch = padded_pairs[:,0]
            gosetB_batch = padded_pairs[:,1]
            
            #permute the data to fit the pytorch transformer
            if torch_vers:
                gosetA_batch = gosetA_batch.permute(1,0,2)
                gosetB_batch = gosetB_batch.permute(1,0,2)
        
            predictions = model(gosetA_batch, gosetB_batch, mask[:,0], mask[:,1]).squeeze(1)
            loss = criterion(predictions, labels)
            acc = binary_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            pred = pred + list(predictions.cpu().data.numpy())
            lab = lab + list(labels.cpu().data.numpy())

    return epoch_loss / len(iterator), epoch_acc / len(iterator), roc_auc_score(lab,pred), lab, pred

In [11]:
def evaluate_cosine(iterator, criterion, torch_vers = False):
    
    epoch_loss = 0
    epoch_acc = 0
    pred = []
    lab = []
    
    for batch in prog_bar(iterator):
            
        #padded pairs: tensor of shape N * 2(protein pair) * L(longest seq) * Emb dim
        padded_pairs = batch[0].to(device)
        labels = batch[1].to(device)
        
        #split data into protA and protB
        gosetA_batch = padded_pairs[:,0]
        gosetB_batch = padded_pairs[:,1]
        
        vector_protA = torch.sum(gosetA_batch, dim = 1)
        vector_protB = torch.sum(gosetB_batch, dim = 1)
        
        cosine = nn.CosineSimilarity(dim=1, eps=1e-6)
        predictions = cosine(vector_protA, vector_protB)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)
            
        epoch_loss += loss.item()
        epoch_acc += acc.item()
            
        pred = pred + list(predictions.cpu().data.numpy())
        lab = lab + list(labels.cpu().data.numpy())

    return epoch_loss / len(iterator), epoch_acc / len(iterator), lab, pred

In [ ]:
model_name = "model.pt"

writer = SummaryWriter(flush_secs=14)
N_EPOCHS = 30
best_roc_val = float('-inf')

for epoch in range(N_EPOCHS):
    

    start_time = time.time()
    train_loss, train_acc, roc_train = train(model, train_grt, optimizer, criterion, torch_vers = False)
    valid_loss, valid_acc, roc_val, _, _ = evaluate(model, val_grt, criterion, torch_vers = False)   
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if best_roc_val < roc_val:
        best_roc_val = roc_val
        torch.save(model.state_dict(),  model_name)

    print_status(epoch, epoch_mins, epoch_secs, train_loss,\
                 train_acc, valid_loss, valid_acc, roc_train, roc_val, optimizer)
    write_scalars_tensorboard(writer, train_loss, valid_loss, train_acc, valid_acc, epoch)

100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:31<00:00,  4.05it/s]


 22%|█████████████████▌                                                               | 28/129 [00:06<00:29,  3.45it/s]

In [ ]:
#WRiTING THE PERFORMANCE ON THE TEST SET #

model = TransformerGO_Scratch(MODEL_SIZE, NR_HEADS, NR_LAYERS, SIZE_FF, DROPOUT)
#model =  GO_Sum_NN(MODEL_SIZE, F_1, F_2, F_3, F_4, DROPOUT)

model.load_state_dict(torch.load(model_name))
model = model.to(device)

with open("training-results.txt", "a") as myfile:
    myfile.write(f"\n ### {model_name} ### \n")
    myfile.write(f"Train set: {len(train_set)}, Valid set: {len(valid_set)}, Test set: {len(test_set)} \n")
    
    valid_loss, valid_acc, roc_val, lab, pred = evaluate(model, test_grt, criterion, torch_vers = False)
    myfile.write(f" \n valid_loss: {valid_loss}, valid_acc: {valid_acc}, roc_val: {roc_val} \n")
    
    valid_loss, valid_acc, lab, pred = evaluate_cosine(test_grt, criterion, torch_vers = False)
    roc_val = metrics.roc_auc_score(lab, pred)
    myfile.write(f" \n ***COSINE*** \n valid_loss: {valid_loss}, valid_acc: {valid_acc}, roc_val: {roc_val} \n")

In [ ]:
#CROSS FOLD VALIDATION EXPERIMENT#

C_FOLD = 5
N_EPOCHS = 20

MODEL_SIZE = EMB_DIM
NR_HEADS = 8
NR_LAYERS = 3
DROPOUT = 0.2
SIZE_FF = 4 * MODEL_SIZE

LR = 0.0001

sz = len(all_dataset)
fold_size = int(sz/C_FOLD)
l = 0
r = fold_size
indexes = np.arange(sz)

val_accs = []
val_rocs = []
wrong_eval = []
wrong_eval_labels = []
for i in range(0, C_FOLD):
    print("Fold nr: ", i, end='\r')
    
    model = TransformerGO_Scratch(MODEL_SIZE, NR_HEADS, NR_LAYERS, SIZE_FF, DROPOUT).to(device)
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    optimizer = optim.Adam(model.parameters(), lr=LR) 
    criterion = nn.BCEWithLogitsLoss().to(device)
    writer = SummaryWriter(flush_secs=14)
    
    val_subset = data.Subset(all_dataset, indexes[l:r])
    c_val_grt = data.DataLoader(val_subset, **params, shuffle = False)
    
    train_subset = data.Subset(all_dataset, np.concatenate([indexes[0:l], indexes[r:sz]]))
    c_train_grt = data.DataLoader(train_subset, **params, shuffle = True)
    
    l += fold_size
    r += fold_size

    best_valid_roc = float('-inf')
    best_valid_acc = float('-inf')
    temp_w_eval = []
    for epoch in range(N_EPOCHS):

        start_time = time.time()
        train_loss, train_acc, roc_train = train(model, c_train_grt, optimizer, criterion)
        valid_loss, valid_acc, roc_val, _, _ = evaluate(model, c_val_grt, criterion)
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
        print_status(epoch, epoch_mins, epoch_secs, train_loss,\
                 train_acc, valid_loss, valid_acc, roc_train, roc_val, optimizer)
        write_scalars_tensorboard(writer, train_loss, valid_loss, train_acc, valid_acc, epoch)
        
        best_valid_roc = max(best_valid_roc, roc_val)
        best_valid_acc = max(best_valid_acc, valid_acc)

    val_rocs.append(best_valid_roc)
    val_accs.append(best_valid_acc)


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:45<00:00,  2.84it/s]


 29%|███████████████████████▊                                                         | 38/129 [00:13<00:36,  2.47it/s]

In [ ]:
#saving the results from the 5 cross validation experiment
with open("5cv_roc_" + neg_path[-24:][:4] + neg_path[-5:] + '.pkl', "wb") as fp:
    pickle.dump(val_rocs, fp)
with open("5cv_acc_" + neg_path[-24:][:4] + neg_path[-5:] + '.pkl', "wb") as fp:
    pickle.dump(val_accs, fp)